In [1]:
import re
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
import spacy

In [3]:
from nltk.stem import WordNetLemmatizer
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/vinay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
# %matplotlib inline

In [23]:
july_data = pd.read_csv("SNOW July Data.csv", encoding = "ISO-8859-1")

In [24]:
july_data = july_data.loc[(july_data['contact'] != 'SPAM SPAM') & (july_data['resolution_code'] != 'Spam')].reset_index(drop=True)

# currently selects descriptions manually. This will later be transitioned to a for-loop or some function
# look at july_data to see which index value you want to see tested
july_samp = july_data.iloc[234]['description']

# DESCRIPTION CLEANING
regex_sol = re.sub(r'https?://\S+', '', july_samp) # removes URL links
regex_sol = re.sub(r"\S*@\S*\s?", "", regex_sol) # removes email accounts
regex_sol = regex_sol.replace("\n", "").replace("< >","").replace("\r", "") # removes newline and <> and \r
regex_sol = re.sub(r"\d+", "", regex_sol) # removes integers 
regex_sol = re.sub(r"([^\s\w]|_)+", " ", regex_sol) # removes non-alphanumeric characters, but maintains whitespace
regex_sol = regex_sol.encode("ASCII", "replace").decode("utf-8").replace("?", " ") # removes all non-ASCII characters
regex_sol = regex_sol.lower()
# regex_sol

# SUBJECT LINE CLEANING
july_samp_2 = july_data.iloc[7]['case']
regex_sol_2 = re.sub(r"[?](CS)[0-9]+", "", july_samp_2)
regex_sol_2 = re.sub(r"\d+", "", regex_sol_2)
regex_sol_2 = re.sub(r"([^\s\w]|_)+", " ", regex_sol_2)
regex_sol_2 = regex_sol_2.lower()
# regex_sol_2

In [25]:
# FUNCTIONS APPLY CURRENT CLEANING STEPS INTO ENTIRE DATASET

def subject_clean(subject_line):
    regex_sol = re.sub(r'https?://\S+', '', subject_line) # removes URL links
    regex_sol = re.sub(r"\S*@\S*\s?", "", regex_sol) # removes email accounts
    regex_sol = regex_sol.replace("\n", "").replace("< >","").replace("\r", "") # removes newline and <> and \r
    regex_sol = re.sub(r"\d+", "", regex_sol) # removes integers 
    regex_sol = re.sub(r"([^\s\w]|_)+", " ", regex_sol) # removes non-alphanumeric characters, but maintains whitespace
    regex_sol = regex_sol.encode("ASCII", "replace").decode("utf-8").replace("?", " ") # removes all non-ASCII characters
    regex_sol = regex_sol.lower() # lower case string
    return regex_sol

def case_line_clean(case_line):
    regex_sol_2 = re.sub(r"[?](CS)[0-9]+", "", case_line) # removes ending "?CS###"
    regex_sol_2 = re.sub(r"\d+", "", regex_sol_2) # removes integers
    regex_sol_2 = re.sub(r"([^\s\w]|_)+", " ", regex_sol_2) # removes non-characters, but maintains whitespace
    regex_sol_2 = regex_sol_2.lower()
    return regex_sol_2

# call the dataset that pertains to you
july_dataset = pd.read_csv("SNOW July Data.csv", encoding = "ISO-8859-1")
june_dataset = pd.read_csv("SNOW June Data.csv", encoding = "ISO-8859-1")

#SPAM case removal
july_dataset = july_dataset.loc[(july_dataset['contact'] != 'SPAM SPAM') & (july_dataset['resolution_code'] != 'Spam')].reset_index(drop=True)
june_dataset = june_dataset.loc[(june_dataset['contact'] != 'SPAM SPAM') & (june_dataset['resolution_code'] != 'Spam')].reset_index(drop=True)

july_dataset['description'] = july_dataset['description'].apply(subject_clean)
june_dataset['description'] = june_dataset['description'].apply(subject_clean)

july_dataset['case'] = july_dataset['case'].apply(case_line_clean)
june_dataset['case'] = june_dataset['case'].apply(case_line_clean)

In [164]:
july_dataset

case  \
0                  add a requisition line  for shopping   
1      enter requisition line  missing from  more ta...   
2      d  room number   e  projects   charge acct s ...   
3                           urgent  change order errors   
4                 urgent  sequencing services in oracle   
...                                                 ...   
1035  your requisition preferences are invalid  fix ...   
1036   national instruments   tmd  document purchase...   
1037   national instruments   tmd  document purchase...   
1038   national instruments   tmd  document purchase...   
1039                         po  incorrect payee set up   

                          contact  \
0                    Janelle Fine   
1                      Alex Clark   
2                  Irina Kufareva   
3                     Kathleen Au   
4                     Kathleen Au   
...                           ...   
1035                    Nina Hill   
1036                        ni ni   
1037   NI  Customer Service Team    
1038   NI  Customer Service Team    
1039              Zachary Roberts   

                                            description  \
0     hello  i need to make a purchase that would ha...   
1      enter requisition line  for buying a specific...   
2     this is report   following cs   d  when i edit...   
3     hello lab members and i have been referring to...   
4     hello we ve noticed that the sequencing servic...   
...                                                 ...   
1035  your requisition preferences are invalid  fix ...   
1036  received from  ni      company name  universit...   
1037  received from  ni      company name  universit...   
1038  received from  ni      company name  universit...   
1039  hi there a po was issued to the supplier  ez b...   

                                      short_description    business_service  \
0                 "Add a Requisition Line" for shopping  Oracle Procurement   
1     "Enter Requisition Line" missing from 'more ta...  Oracle Procurement   
2     (d) room number, (e) projects & charge acct's,...  Oracle Procurement   
3                         (Urgent!) Change Order Errors  Oracle Procurement   
4               (Urgent!) Sequencing Services in Oracle  Oracle Procurement   
...                                                 ...                 ...   
1035  Your requisition preferences are invalid. Fix ...  Oracle Procurement   
1036  [National Instruments] (TMD-2614068) Document ...  Oracle Procurement   
1037  [National Instruments] (TMD-2614068) Document ...  Oracle Procurement   
1038  [National Instruments] (TMD-2614068) Document ...  Oracle Procurement   
1039                        [PO] Incorrect Payee Set-up   Supplier Invoices   

               service_offering                      resolution_code  
0                   How to Shop  Inconclusive - No Customer Response  
1                   How to Shop  Inconclusive - No Customer Response  
2                   How to Shop                                  NaN  
3          Other Buying Inquiry                                  NaN  
4                     Punchouts                                  NaN  
...                         ...                                  ...  
1035  Approvals on Requisitions                                  NaN  
1036       Other Buying Inquiry  Inconclusive - No Customer Response  
1037       Other Buying Inquiry  Inconclusive - No Customer Response  
1038       Other Buying Inquiry  Inconclusive - No Customer Response  
1039    Incorrect Bill/Invoices  Inconclusive - No Customer Response  

[1040 rows x 7 columns]

In [165]:
june_dataset

case  \
0     revised purchase order needed     document pu...   
1     revised purchase order needed   document purc...   
2                                      orders received   
3               d hubs vendor setup in payment compass   
4                                                        
..                                                 ...   
447                                zip files in emails   
448   national instruments   tmd  document purchase...   
449   national instruments   tmd  document purchase...   
450   national instruments   tmd  document purchase...   
451             ticket  re  document purchase order      

                               contact  \
0                       Lori  Bourgery   
1                       Lori  Bourgery   
2                       Melissa Taylor   
3                            Leah Kent   
4                      beckman beckman   
..                                 ...   
447                    Stewart Pickett   
448         NI  Customer Service Team    
449                              ni ni   
450         NI  Customer Service Team    
451  ga international ga international   

                                           description  \
0    received from  good morning please forward a r...   
1    received from  good morning please forward a c...   
2    received from  good morning we received  order...   
3    will the team please let me know if d hubs has...   
4    received from  hello thank you for your order ...   
..                                                 ...   
447  received from  hello i received the attached f...   
448  received from  ni      company name  universit...   
449  received from  ni      dear customer we haven ...   
450  received from  ni       thank you for your rec...   
451  received from  we ve received your request tha...   

                                     short_description    business_service  \
0    **Revised Purchase Order Needed**    Document ...  Oracle Procurement   
1    **Revised Purchase Order Needed**  Document Pu...  Oracle Procurement   
2                                 2018 Orders Received  Oracle Procurement   
3              3D HUBS Vendor Setup in Payment Compass  Oracle Procurement   
4                                             91693761  Oracle Procurement   
..                                                 ...                 ...   
447                                Zip Files In Emails  Oracle Procurement   
448  [National Instruments] (TMD-2614068) Document ...  Oracle Procurement   
449  [National Instruments] (TMD-2614069) Document ...  Oracle Procurement   
450  [National Instruments] (TMD-2614075) Document ...  Oracle Procurement   
451  [Ticket#19100681] RE: Document Purchase Order ...  Oracle Procurement   

         service_offering                      resolution_code  
0    Other Buying Inquiry                                  NaN  
1    Other Buying Inquiry  Inconclusive - No Customer Response  
2    Other Buying Inquiry                                  NaN  
3    Other Buying Inquiry                                  NaN  
4    Other Buying Inquiry  Inconclusive - No Customer Response  
..                    ...                                  ...  
447  Other Buying Inquiry  Inconclusive - No Customer Response  
448  Other Buying Inquiry  Inconclusive - No Customer Response  
449  Other Buying Inquiry  Inconclusive - No Customer Response  
450  Other Buying Inquiry  Inconclusive - No Customer Response  
451  Other Buying Inquiry  Inconclusive - No Customer Response  

[452 rows x 7 columns]

In [166]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
domain_stop_words = ['hi', 'hello', 'thank', 'thanks', 'com', 'the', 're', 'php', 'http', 'XXXXXXXXX', 'would', 'however', 'please', 'do', 'can', 'may', 'check', 'pende', 'tell', 'use', 'call', 'let', 'dear', 'see', 'click', 'still', 'unable', 'even', 'minute', 'basically', 'seem', 'expect', 'pcie', 'usd', 'go', 'could', 'advise', 'appreciate', 'regard', 'also', 'end', 'sure', 'copy', 'phone', 'know', 'accidently', 'reply', 'web', 'soon', 'regard']
# unable, seem
stop_words.extend(domain_stop_words)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [167]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [168]:
julydescrip = list(july_dataset['description'])

In [169]:


data_words = list(sent_to_words(julydescrip))

In [170]:
print(data_words[:1])

[['hello', 'need', 'to', 'make', 'purchase', 'that', 'would', 'have', 'used', 'minirequest', 'for', 'in', 'marketplace', 'the', 'buying', 'and', 'paying', 'training', 'class', 'shows', 'that', 'this', 'is', 'done', 'in', 'oracle', 'by', 'choosing', 'add', 'requisition', 'line', 'with', 'an', 'arrow', 'pointing', 'towards', 'the', 'more', 'tasks', 'dropdown', 'on', 'the', 'main', 'procurements', 'page', 'however', 'my', 'dropdown', 'for', 'more', 'tasks', 'does', 'not', 'have', 'choice', 'for', 'adding', 'requisition', 'line', 'and', 'cannot', 'find', 'it', 'elsewhere', 'in', 'the', 'procurement', 'system', 'can', 'you', 'please', 'direct', 'me', 'to', 'where', 'can', 'find', 'this', 'option', 'to', 'place', 'an', 'order', 'for', 'something', 'from', 'company', 'who', 'does', 'not', 'have', 'punchout', 'thanks']]


In [171]:
#initiate blank list
text_words = []

#for loop to tokenize all words in
for sentence in julydescrip:
    temp = gensim.utils.simple_preprocess(sentence)
    text_words.append(temp)

In [172]:
# Build the bigram
bigram = gensim.models.Phrases(text_words, min_count=5, threshold=100)

In [173]:
#Removeal of stop words
text_words_no_stops = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text_words]

#Make texts into bigram model
text_words_no_stops_bigram = [bigram[doc] for doc in text_words_no_stops]

In [174]:
text_words_no_stops_bigram[0]

['need',
 'make',
 'purchase',
 'used',
 'minirequest',
 'marketplace',
 'buying',
 'paying',
 'training',
 'class',
 'shows',
 'done',
 'oracle',
 'choosing',
 'add',
 'requisition',
 'line',
 'arrow',
 'pointing',
 'towards',
 'tasks',
 'dropdown',
 'main',
 'procurements',
 'page',
 'dropdown',
 'tasks',
 'choice',
 'adding',
 'requisition',
 'line',
 'cannot',
 'find',
 'elsewhere',
 'procurement',
 'system',
 'direct',
 'find',
 'option',
 'place',
 'order',
 'something',
 'company',
 'punchout']

In [175]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#lemmatizer.lemmatize('rocks')

In [176]:
# word_list= text_words_no_stops_bigram[0]
# w2 = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
# w2

In [177]:
"""
for cases in text_words_no_stops_bigarm:
    for words in cases:
        print(words)
        print ("{0:20}{1:20}".format(words,lemmatizer.lemmatize(words, pos="n")))
"""

'\nfor cases in text_words_no_stops_bigarm:\n    for words in cases:\n        print(words)\n        print ("{0:20}{1:20}".format(words,lemmatizer.lemmatize(words, pos="n")))\n'

In [178]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet

# return POS tag for next function
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [179]:
# Lemmatizing function 
# takes in full list of descriptions with stop words removed
# returns descriptions lemmatized
def lemm(text_stop_no_bigram):
    count = 0
    out = []
    for descrip in text_stop_no_bigram:
        count += 1
        inner = []
        for word in descrip:
            pos = get_wordnet_pos(word)
            inner.append(lemmatizer.lemmatize(word, pos))
        out.append(inner)
    return out


In [180]:
# Lemmatizes 
lemmed = (lemm(text_words_no_stops_bigarm))
#lemmed[0]

In [181]:
#test = [['have', 'runs', 'what', 'oracle', 'requisitioning']]
#lemm(test)

In [182]:
#text_words_no_stops_bigram[0]

In [183]:
"""
#Function for lemmatization due to many steps
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
"""

'\n#Function for lemmatization due to many steps\ndef lemmatization(texts, allowed_postags=[\'NOUN\', \'ADJ\', \'VERB\', \'ADV\']):\n    texts_out = []\n    for sent in texts:\n        doc = nlp(" ".join(sent)) \n        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])\n    return texts_out\n'

In [184]:
#getting spacy 'en' and lemmetizing bi-grams
# nlp = spacy.load('en', disable=['parser', 'ner'])
# data_lemmatized = lemmatization(text_words_no_stops_bigram, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [185]:
#print(data_lemmatized[0])

In [186]:
#Create Dictionary
id2word = gensim.corpora.Dictionary(lemmed)

#Create Corpus
texts = lemmed

#Build TDF
corpus = [id2word.doc2bow(text) for text in texts]

corpus[:1]

[[(0, 2),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 2),
  (11, 1),
  (12, 2),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 2),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 1),
  (36, 1)]]

In [187]:
#Read Corups/TDF per case
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

, 2),
  ('pur', 1),
  ('confirmation', 1),
  ('contact', 1),
  ('day', 1),
  ('question', 1),
  ('receive', 2),
  ('shipping', 1),
  ('receipt', 1),
  ('feel_free', 1),
  ('process', 1),
  ('time', 2),
  ('appreciate', 1),
  ('greatly', 1),
  ('soon', 1),
  ('document', 1),
  ('message', 1),
  ('ship', 1),
  ('week', 2),
  ('ucsd', 2),
  ('pdf', 2),
  ('possible', 1),
  ('notification', 1),
  ('make_sure', 1),
  ('last', 2),
  ('say', 1),
  ('tell', 1),
  ('issue', 1),
  ('multiple', 1),
  ('much', 1),
  ('chemical', 2),
  ('financials', 3),
  ('morning', 1),
  ('fyi', 1),
  ('req_implemented', 1),
  ('quick', 1),
  ('left', 1),
  ('joseph', 1),
  ('crunch', 1),
  ('discus', 1),
  ('ihad', 1),
  ('ihaven', 1),
  ('iwant', 1),
  ('puralso', 1),
  ('sawires', 1),
  ('soif', 1),
  ('supplieryet', 1),
  ('throughout', 1)],
 [('need', 3),
  ('training', 1),
  ('work', 1),
  ('create', 3),
  ('edit', 1),
  ('understand', 1),
  ('po', 3),
  ('service', 1),
  ('year', 1),
  ('person', 1),
  ('

In [188]:
# Build LDA Model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=33,
                                           alpha='auto',
                                           per_word_topics=True)



In [189]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.050*"item" + 0.028*"request" + 0.019*"company" + 0.019*"punchout" + '
  '0.016*"add" + 0.015*"system" + 0.015*"marketplace" + 0.014*"ni" + '
  '0.014*"oracle" + 0.014*"unable"'),
 (1,
  '0.046*"cost" + 0.030*"tell" + 0.020*"transfer" + 0.020*"packing_slip" + '
  '0.012*"takako" + 0.012*"hop" + 0.012*"matter" + 0.011*"actri" + '
  '0.010*"value" + 0.010*"accidentally"'),
 (2,
  '0.029*"deliver" + 0.024*"miss" + 0.019*"want" + 0.019*"location" + '
  '0.018*"delivery" + 0.016*"found" + 0.014*"pending_approval" + 0.013*"lot" + '
  '0.013*"something" + 0.012*"wait"'),
 (3,
  '0.072*"order" + 0.031*"email" + 0.024*"receive" + 0.021*"number" + '
  '0.021*"item" + 0.015*"po" + 0.014*"ship" + 0.014*"address" + '
  '0.014*"invoice" + 0.013*"price"'),
 (4,
  '0.031*"oracle" + 0.027*"order" + 0.025*"need" + 0.024*"po" + '
  '0.023*"requisition" + 0.016*"try" + 0.015*"know" + 0.014*"use" + '
  '0.013*"lab" + 0.013*"get"'),
 (5,
  '0.028*"email" + 0.022*"service" + 0.021*"contact" + 0.019*

In [190]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.896744712365961

Coherence Score:  nan


In [191]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.152339  0.301778       1        1  28.336542
3      0.219555 -0.083222       2        1  18.173790
9      0.171703 -0.135821       3        1  15.680714
5      0.087615 -0.073607       4        1   9.666205
7      0.093363 -0.117245       5        1   9.158220
0     -0.031411  0.200433       6        1   7.082974
8     -0.231504 -0.059864       7        1   6.707171
2     -0.183859 -0.014880       8        1   2.826494
6     -0.124075  0.000528       9        1   1.454970
1     -0.153726 -0.018099      10        1   0.912912, topic_info=          Term        Freq       Total Category  logprob  loglift
461   document  324.000000  324.000000  Default  30.0000  30.0000
483    message  412.000000  412.000000  Default  29.0000  29.0000
39        item  312.000000  312.000000  Default  28.0000  28.0000
399        usd  221.000000  221.000000  Default  27.0000  27.0000
28    purchase  409.000000  409.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2608  invoiced    1.540583    2.404456  Topic10  -5.5403   4.2511
668        coa    3.265056    5.137881  Topic10  -4.7892   4.2429
908      value    3.889745   36.189747  Topic10  -4.6142   2.4659
4350     stuff    1.900120    8.772482  Topic10  -5.3306   3.1666
865   jolla_ca    1.804381   11.460403  Topic10  -5.3823   2.8476

[484 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
406       1  0.982809              able
3799      5  0.956633  accept_liability
1164     10  0.842664      accidentally
45        1  0.306551           account
45        2  0.467125           account
...     ...       ...               ...
553       8  0.913841            wonder
43        1  0.988009              work
448       4  0.984432             write
1234      3  0.984553               www
912      10  0.792420      yvonne_huynh

[575 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 10, 6, 8, 1, 9, 3, 7, 2])